In [1]:
import pandas as pd
from nba_api.stats.endpoints import leaguedashplayerstats

import requests

# Step 1: Read the URL from the gitignored file
url_file = "download_url.txt"
try:
    with open(url_file, "r") as file:
        url = file.readline().strip()  # Read and strip any extra whitespace
except FileNotFoundError:
    print(f"Error: {url_file} not found. Please create the file and add the URL.")
    exit()

# Step 2: Download the file using the URL
filename = "lebron_link25.csv"  # Name to save the file
try:
    response = requests.get(url)
    response.raise_for_status()  # Check for request errors

    # Save the content to a local file
    with open(filename, "wb") as file:
        file.write(response.content)
    print(f"File downloaded and saved as {filename}")
except requests.exceptions.RequestException as e:
    print(f"Error downloading the file: {e}")

df=pd.read_csv('lebron_link25.csv')
print(df.columns)
print(len(df))
df['year']=2025
df
df.dropna(subset=['Name'],inplace=True)
old_df=pd.read_csv('lebron.csv')
old_df=old_df[old_df.year<2025]
old_df
column_mapping = {
    'Season': 'Season',
    'Name': 'Player',
    'Team': 'team',
    'Minutes': 'Minutes',
    'Position': 'Pos',
    'offensiveArchetype': 'Offensive Archetype',
    'LEBRON_WAR': 'WAR',
    'LEBRON': 'LEBRON',
    'OLEBRON': 'O-LEBRON',
    'DLEBRON': 'D-LEBRON',
    'defensive_role': 'Defensive Role',
    'PLAYER_ID': 'NBA ID',
    'G': 'Games',
    # 'bref_id' and 'Age' omitted intentionally
}
df.rename(columns=column_mapping,inplace=True)

samecolumns =set(old_df.columns) - (set(old_df.columns)-set(df.columns))
samecolumns
df=df[list(samecolumns)]
print(old_df.columns)

def update(year):
    season_string = str(year-1) +'-'+str(year)[-2::]
    print(season_string)
    frames = leaguedashplayerstats.LeagueDashPlayerStats(season=season_string).get_data_frames()
    df = frames[0]

    return dict(zip(df['PLAYER_ID'],df['AGE']))
ages = update(2025)
df['Age']=df['NBA ID'].map(ages)
idframe=pd.read_csv('index_master.csv')
id_map = dict(zip(idframe['nba_id'],idframe['bref_id']))

df['bref_id']=df['NBA ID'].map(id_map)
df['Player']=df['Player'].str.lower()

newframe = pd.concat([old_df,df])
newframe.to_csv('lebron.csv',index=False)
newframe.to_csv('../web_app/data/lebron.csv',index=False)

newframe

File downloaded and saved as lebron_link25.csv
Index(['Season', 'ThroughDate', 'PLAYER_ID', 'Name', 'player_name', 'Team',
       'Position', 'G', 'GS', 'Minutes', 'advancedPosition',
       'offensiveArchetype', 'defensive_role', 'LEBRON', 'OLEBRON', 'DLEBRON',
       'LEBRON_WAR', 'boxLEBRON', 'boxOLEBRON', 'boxDLEBRON', 'multiOLEBRON',
       'multiDLEBRON', 'game_date', 'RegularSeasonCareerGameNumber',
       'last3SeasonsGames', 'maxYOS', 'projectedMinutes', 'multiLEBRON',
       'projOLEBRON', 'projDLEBRON', 'projLEBRON', 'Matchup_Difficulty',
       'pad_true_usage_pct', 'two_way_usage'],
      dtype='object')
520
Index(['Season', 'Player', 'team', 'Minutes', 'Pos', 'Offensive Archetype',
       'WAR', 'LEBRON', 'O-LEBRON', 'D-LEBRON', 'year', 'Defensive Role',
       'NBA ID', 'bref_id', 'Games', 'Age'],
      dtype='object')
2024-25


,Season,Player,team,Minutes,Pos,Offensive Archetype,WAR,LEBRON,O-LEBRON,D-LEBRON,year,Defensive Role,NBA ID,bref_id,Games,Age
0,2009-10,lebron james,CLE,2966.000000,SF,Shot Creator,20.570000,8.840000,7.310000,1.530000,2010,NaN,2544.0,jamesle01,76.0,25.0
1,2009-10,rodrigue beaubois,DAL,700.000000,PG,Primary Ball Handler,0.640000,-0.930000,0.090000,-1.030000,2010,NaN,201958.0,beaubro01,56.0,21.0
2,2009-10,david andersen,HOU,891.000000,C,Post Scorer,0.800000,-0.950000,-0.970000,0.020000,2010,NaN,2431.0,anderda03,63.0,29.0
3,2009-10,eric gordon,LAC,2229.000000,SG,Secondary Ball Handler,2.010000,-0.950000,1.140000,-2.090000,2010,NaN,201569.0,gordoer01,62.0,21.0
4,2009-10,michael beasley,MIA,2328.000000,PF,Athletic Finisher,2.030000,-0.990000,-0.990000,0.000000,2010,NaN,201563.0,beaslmi01,78.0,21.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
514,2024-25,jalen wilson,BRK,1020.668333,PF,Stationary Shooter,-0.523248,-3.263948,-1.856605,-1.407343,2025,Helper,1630592.0,wilsoja03,37.0,24.0
515,2024-25,corey kispert,WAS,863.016667,SF,Off Screen Shooter,-0.471400,-3.351505,-1.448493,-1.903012,2025,Chaser,1630557.0,kispeco01,31.0,25.0
516,2024-25,bilal coulibaly,WAS,1098.693333,SF,Secondary Ball Handler,-0.635420,-3.400918,-2.425736,-0.975182,2025,Point of Attack,1641731.0,coulibi01,32.0,20.0
517,2024-25,josh green,CHO,963.101667,SG,Stationary Shooter,-0.608752,-3.459278,-3.388340,-0.070937,2025,Point of Attack,1630182.0,greenjo02,33.0,24.0
